Install the tweepy library

In [1]:
# !pip install tweepy

Import required libraries

In [2]:
import os
import tweepy as tw
import pandas as pd

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
for tweet in tweets:
    print(tweet.text)

RT @martinmckee: We need clear &amp; simple messaging on #COVID - @SimpleCovid are doing a great job. Thanks folks https://t.co/GL5ZOXykof
RT @humour007: Working in zomato from 8-8 but 4k only salary in lockdown.. lots of loans in lockdown. Friends please help my family from th…
Today's Covid positive. #StaySafeStayHome 
#CoronaVirusUpdates #COVID https://t.co/tibJkafrTk
Haven’t the #publicHeath community been saying from the beginning? Just look at life now in those countries in East… https://t.co/246VZRPwbG
RT @bitcoinconnect: Black Friday 2020: The best PS4 and PS5 deals right now SEE MORE HERE ==&gt; https://t.co/8PTOFMzQGs  #nintendoswitch #Xbo…


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

[]

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

["Today's Covid positive. #StaySafeStayHome \n#CoronaVirusUpdates #COVID https://t.co/tibJkafrTk",
 'Haven’t the #publicHeath community been saying from the beginning? Just look at life now in those countries in East… https://t.co/246VZRPwbG',
 'Restaurant staffers shed pounds under chef-prescribed diet SEE MORE HERE ==&gt; https://t.co/HOZ3688qLE… https://t.co/qJxa8UEGJk',
 'In technical and vocational education and training systems, vulnerabilities including low levels of digitalization… https://t.co/GsRRll5Hl0',
 'Scalper-Bots Shake Down Desperate PS5, Xbox Series X Shoppers SEE MORE HERE ==&gt; https://t.co/8PTOFMzQGs… https://t.co/LhgL88Wxhs']

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [18]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [19]:
users_locs

[]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location"])

AssertionError: 2 columns passed, passed data had 3 columns

In [ ]:
tweet_df